In [ ]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "trl" "bitsandbytes==0.40.2" "einops"

**Dataset Processing**

In [18]:
INTRO_BLURB = "Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "datafrmae columns:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [16]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/padnas_code.csv')
df.head()

,Dataset,DataFrame,Columns,Task,Steps,ExpectedCode
0,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Calculate the average unit price for each item.,"Group by 'ItemCode' or 'ItemName'., Compute th...",avg_price_per_item = df.groupby('ItemCode').ag...
1,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Find the total quantity sold for each item.,"Group by 'ItemCode' or 'ItemName'., Sum the 'Q...",total_quantity_per_item = df.groupby('ItemCode...
2,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Identify the date with the highest sales amount.,"Group by 'InvoiceDocDate'., Sum the 'TotalAmou...",sales_per_date = df.groupby('InvoiceDocDate')....
3,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Compute the total tax collected from sales.,Sum the 'Tax' column.,total_tax_collected = df['Tax'].sum()
4,Sales dataset,df,"InvoiceDocDate, CustomerCode, CustomerName, It...",Determine the item that has been sold to the m...,"Group by 'ItemCode' or 'ItemName'., Count the ...",unique_customers_per_item = df.groupby('ItemCo...


In [17]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Dataset', 'DataFrame', 'Columns', 'Task', 'Steps', 'ExpectedCode'],
    num_rows: 60
})

In [ ]:
def apply_prompt_template(examples):
  instruction = examples['Task']
  context = examples.get("Columns")
  response = examples["ExpectedCode"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
  else:
    full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


In [20]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Calculate the average unit price for each item.

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
avg_price_per_item = df.groupby('ItemCode').agg(AverageUnitPrice=('UnitPrice', 'mean'))

### End


**Finetune model**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [23]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [24]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [27]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [28]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)


In [ ]:
trainer.train()

**Testing**

In [30]:
from transformers import pipeline

In [31]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [32]:
prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Top 5 items in term of sales money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
Top 5 items in term of sales money

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
sales_by_item_amount = df.groupby('ItemCode').agg(Sales=('UnitPrice','sum'))
sales_by_item_percentage = df.groupby('ItemCode').agg(SalesPercentage=('UnitPrice / Quantity', 'divide'))
sales_by_item_discount = df.groupby('ItemCode').agg(DiscountPercentage=('Tax / Quantity', 'divide'))
sales_by_item_tax = df.groupby('ItemCode').agg(TaxAmount=('Tax / Quantity','mul'))
sales_by_item_total_amount = df.groupby('ItemCode').agg(TotalAmount=('Quantity * UnitPrice','mul'))
sales
None


In [33]:
prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 products has higest discounts

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
top 5 products has higest discounts

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
discount_per_item = df.groupby('ItemCode').agg(Discount=('Discount','mean')).sort_values(by='Discount', ascending=False)
top_discount_items = discount_per_item.head(5)


None


In [34]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
how many unique items are there

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Below is an instruction that describes a task. Write python pandas code response that appropriately completes the request.

### Instruction:
how many unique items are there

datafrmae columns:
InvoiceDocDate, CustomerCode, CustomerName, ItemCode, ItemName, UnitPrice, Quantity, Discount, Tax, TotalAmount

### Response:
unique_items = df.groupby('CustomerCode').size()


None
